# DiD Analysis: MHDI (2000 / 2010)

**Author**: elia.matsumoto@fgv.br (2026)

## Overview
This notebook performs a Difference-in-Differences (DiD) analysis on the Municipal Human Development Index (MHDI) data for years 2000 and 2010.

## Research Design
- **Treatment Group**: Municipalities receiving intervention
- **Control Group**: Municipalities not receiving intervention
- **Pre-period**: 2000
- **Post-period**: 2010

## Input File
- `Data_1_MHDI.xlsx:
  - `Subprefecture`: List of São Paulo subprefectures
  - `CEU`: CEU facility data including opening dates
  - `AFTER`: Time dummy (0=2000, 1=2010)
  - `TREAT`: Treatment dummy (1=treatment, 0=control)
  - `GEN`: General MHDI
  - `INC`: Income component
  - `LONG`: Longevity component
  - `EDU`: Education component

## Output Files
1. `Fig_1_MHDI_Descriptive_Statistics.jpeg`: descritive statisticas
2. `Fig_1_MHDI_DiD_Confidence.jpeg`: confidence intervals
3. `Fig_1_MHDI_DiD_Parallel_Trends.jpeg`: parallel trends
4. `Results_1_MDHI_DiD.xlsx`: regression models results



## 1. Setup and Configuration

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pathlib import Path
import warnings

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("colorblind")

# Display settings
pd.set_option('display.float_format', lambda x: f'{x:.4f}')
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

print("Libraries imported successfully.")

## 2. Data Loading and Validation

In [ ]:
# Configuration
DATA_FILE = 'Data_1_MHDI.xlsx'
DATA_SHEET = 'Sheet1'

def load_and_validate_data(file_path, sheet_name):
    """
    Load and validate the MHDI dataset
    """
    try:
        # Check if file exists
        if not Path(file_path).exists():
            raise FileNotFoundError(f"File '{file_path}' not found.")
        
        # Load data
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        
        print(f"Data loaded successfully. Shape: {df.shape}")
        print(f"Columns: {list(df.columns)}")
        
        # Required columns for DiD analysis
        required_columns = ['GEN', 'INC', 'LONG', 'EDU', 'AFTER', 'TREAT']
        missing_columns = [col for col in required_columns if col not in df.columns]
        
        if missing_columns:
            raise ValueError(f"Missing required columns: {missing_columns}")
        
        # Check for missing values
        missing_values = df[required_columns].isnull().sum()
        if missing_values.any():
            print(f"\nMissing values found:\n{missing_values[missing_values > 0]}")
        else:
            print("\nNo missing values in required columns.")
        
        # Basic statistics
        print("\nBasic statistics:")
        print(df[required_columns].describe().T)
        
        # Check treatment/control balance
        print("\nTreatment/Control distribution:")
        print(df.groupby('AFTER')['TREAT'].value_counts().unstack())
        
        return df
        
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

# Load data
df = load_and_validate_data(DATA_FILE, DATA_SHEET)

## 3. Exploratory Data Analysis

In [ ]:
def plot_descriptive_stats(df):
    """
    Create descriptive plots for the MHDI data
    """
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle('MHDI Descriptive Statistics', fontsize=16, y=1.02)
    
    # Plot 1: Distribution of MHDI components
    components = ['GEN', 'INC', 'LONG', 'EDU']
    for idx, component in enumerate(components):
        ax = axes[idx // 2, idx % 2]
        df[component].hist(ax=ax, bins=30, edgecolor='black')
        ax.set_title(f'Distribution of {component}')
        ax.set_xlabel(component)
        ax.set_ylabel('Frequency')
    
    # Plot 5: Treatment vs Control groups
    ax = axes[1, 2]
    treatment_counts = df['TREAT'].value_counts()
    ax.bar(['Control', 'Treatment'], treatment_counts.values)
    ax.set_title('Treatment vs Control Groups')
    ax.set_ylabel('Number of Municipalities')
    
    plt.tight_layout()
    # Save plot
    Fig_File = 'Fig_1_MHDI_Descriptive_Statistics.jpeg'
    plt.savefig(Fig_File, format='jpeg', dpi=300)
    plt.show()
    
# Plot descriptive statistics
if df is not None:
    plot_descriptive_stats(df)

## 4. DiD Analysis Functions

In [ ]:
def run_did_regression(df, outcome_var, formula=None, cluster_var=None):
    """
    Run a Difference-in-Differences regression
    
    Parameters:
    -----------
    df : DataFrame
        Input data
    outcome_var : str
        Outcome variable name
    formula : str, optional
        Custom formula. If None, uses default DiD formula
    cluster_var : str, optional
        Variable to cluster standard errors
    
    Returns:
    --------
    result : RegressionResults object
    """
    if formula is None:
        formula = f'{outcome_var} ~ AFTER + TREAT + AFTER * TREAT'
    
    try:
        model = smf.ols(formula=formula, data=df)
        
        if cluster_var and cluster_var in df.columns:
            # Use clustered standard errors
            result = model.fit(cov_type='cluster', cov_kwds={'groups': df[cluster_var]})
        else:
            result = model.fit()
        
        return result
        
    except Exception as e:
        print(f"Error running regression for {outcome_var}: {e}")
        return None

def extract_did_results(results_dict):
    """
    Extract key DiD results into a summary DataFrame
    """
    summary_data = []
    
    for outcome, result in results_dict.items():
        if result is None:
            continue
            
        # Extract the interaction term (DiD estimator)
        if 'AFTER:TREAT' in result.params:
            did_coef = result.params['AFTER:TREAT']
            did_se = result.bse['AFTER:TREAT']
            did_pval = result.pvalues['AFTER:TREAT']
            
            # Calculate confidence intervals
            conf_int = result.conf_int().loc['AFTER:TREAT']
            
            summary_data.append({
                'Outcome': outcome,
                'DiD Coefficient': did_coef,
                'Std. Error': did_se,
                'P-value': did_pval,
                '95% CI Lower': conf_int[0],
                '95% CI Upper': conf_int[1],
                'Significant': 'Yes' if did_pval < 0.05 else 'No',
                'Observations': result.nobs,
                'R-squared': result.rsquared
            })
    
    return pd.DataFrame(summary_data)

## 5. Run DiD Analysis

In [ ]:
if df is not None:
    # Define outcomes to analyze
    outcomes = {
        'General MHDI': 'GEN',
        'Income Component': 'INC',
        'Longevity Component': 'LONG',
        'Education Component': 'EDU'
    }
    
    # Run DiD regressions
    results = {}
    
    print("Running DiD Analysis...")
    print("=" * 60)
    
    for label, var in outcomes.items():
        print(f"\n{label} ({var}):")
        print("-" * 40)
        
        result = run_did_regression(df, var)
        results[label] = result
        
        if result is not None:
            # Display key results
            print(f"DiD Coefficient (AFTER:TREAT): {result.params.get('AFTER:TREAT', 'N/A'):.4f}")
            print(f"P-value: {result.pvalues.get('AFTER:TREAT', 'N/A'):.4f}")
            print(f"Observations: {result.nobs:,}")
            print(f"R-squared: {result.rsquared:.4f}")
            
            # Store full summary in a variable for later inspection
            globals()[f'result_{var}'] = result
    
    # Create summary table
    print("\n" + "=" * 60)
    print("MHDI: SUMMARY OF DiD RESULTS")
    print("=" * 60)
    
    summary_df = extract_did_results(results)
    if not summary_df.empty:
        print("\nSummary Table:")
        print(summary_df.to_string(index=False))
else:
    print("Data not available. Please check data loading step.")

## 6. Visualization of DiD Results

In [ ]:
def plot_did_results(summary_df):
    """
    Create visualization of DiD results
    """
    if summary_df.empty:
        print("No results to visualize.")
        return
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Plot 1: DiD coefficients with confidence intervals
    ax1 = axes[0]
    
    # Sort by coefficient value
    plot_df = summary_df.sort_values('DiD Coefficient')
    
    y_pos = np.arange(len(plot_df))
    ax1.barh(y_pos, plot_df['DiD Coefficient'], xerr=[
        plot_df['DiD Coefficient'] - plot_df['95% CI Lower'],
        plot_df['95% CI Upper'] - plot_df['DiD Coefficient']
    ], capsize=5, alpha=0.7)
    
    ax1.set_yticks(y_pos)
    ax1.set_yticklabels(plot_df['Outcome'])
    ax1.set_xlabel('DiD Coefficient')
    ax1.set_title('DiD Coefficients with 95% Confidence Intervals')
    ax1.axvline(x=0, color='red', linestyle='--', alpha=0.5)
    
    # Add significance markers
    for i, (_, row) in enumerate(plot_df.iterrows()):
        if row['Significant'] == 'Yes':
            ax1.text(row['DiD Coefficient'] * 1.1, i, '*', 
                    fontsize=14, color='red', va='center')
    
    # Plot 2: P-values
    ax2 = axes[1]
    colors = ['red' if p < 0.05 else 'blue' for p in plot_df['P-value']]
    ax2.barh(y_pos, -np.log10(plot_df['P-value']), color=colors, alpha=0.7)
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels([])  # Hide y labels as they're in first plot
    ax2.set_xlabel('-log10(P-value)')
    ax2.set_title('Statistical Significance (-log10 P-value)')
    ax2.axvline(x=-np.log10(0.05), color='red', linestyle='--', alpha=0.5)
    ax2.text(-np.log10(0.05), len(plot_df)-0.5, 'p=0.05', 
             ha='right', va='bottom', color='red', fontsize=10)
    
    plt.tight_layout()
   
    # Print interpretation
    print("\nInterpretation of DiD Coefficients:")
    print("-" * 40)
    for _, row in summary_df.iterrows():
        sig = "(statistically significant)" if row['Significant'] == 'Yes' else "(not statistically significant)"
        print(f"{row['Outcome']}: {row['DiD Coefficient']:.4f} {sig}")

    # Save plot
    Fig_File = 'Fig_1_MHDI_DiD_Confidence.jpeg'
    plt.savefig(Fig_File, format='jpeg', dpi=300)
    plt.show()


# Plot results
if 'summary_df' in locals() and not summary_df.empty:
    plot_did_results(summary_df)

## 7. Parallel Trends Visualization

In [ ]:
def plot_parallel_trends(df, outcome_vars):
    """
    Plot parallel trends for visual inspection
    """
    if df is None or len(df) == 0:
        return
    
    n_plots = len(outcome_vars)
    n_cols = 2
    n_rows = (n_plots + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4 * n_rows))
    axes = axes.flatten() if n_plots > 1 else [axes]
    
    for idx, (label, var) in enumerate(outcome_vars.items()):
        if idx >= len(axes):
            break
            
        ax = axes[idx]
        
        # Calculate group means
        group_means = df.groupby(['AFTER', 'TREAT'])[var].mean().unstack()
        
        # Plot lines
        time_labels = ['Pre (2000)', 'Post (2010)']
        x_pos = [0, 1]
        
        ax.plot(x_pos, group_means[0].values, 'o-', label='Control', linewidth=2, markersize=8)
        ax.plot(x_pos, group_means[1].values, 's-', label='Treatment', linewidth=2, markersize=8)
        
        ax.set_xticks(x_pos)
        ax.set_xticklabels(time_labels)
        ax.set_ylabel(label)
        ax.set_title(f'Parallel Trends: {label}')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        # Annotate the DiD effect
        did_effect = (group_means[1].iloc[1] - group_means[1].iloc[0]) - \
                     (group_means[0].iloc[1] - group_means[0].iloc[0])
        ax.annotate(f'DiD = {did_effect:.3f}', 
                   xy=(0.5, 0.95), xycoords='axes fraction',
                   ha='center', va='top',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Hide empty subplots
    for idx in range(len(outcome_vars), len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    # Save plot
    Fig_File = 'Fig_1_MHDI_DiD_Parallel_Trends.jpeg'
    plt.savefig(Fig_File, format='jpeg', dpi=300)
    plt.show()

# Plot parallel trends
if df is not None:
    outcomes = {
        'General MHDI': 'GEN',
        'Income': 'INC',
        'Longevity': 'LONG',
        'Education': 'EDU'
    }
    plot_parallel_trends(df, outcomes)

## 8. Robustness Checks (Optional)

In [ ]:
def run_robustness_checks(df):
    """
    Run additional robustness checks
    """
    if df is None:
        return
    
    print("Running Robustness Checks...")
    print("=" * 60)
    
    # Check 1: Pre-treatment means
    print("\n1. Pre-treatment Balance:")
    pre_data = df[df['AFTER'] == 0]
    
    for var in ['GEN', 'INC', 'LONG', 'EDU']:
        control_mean = pre_data[pre_data['TREAT'] == 0][var].mean()
        treat_mean = pre_data[pre_data['TREAT'] == 1][var].mean()
        diff = treat_mean - control_mean
        print(f"   {var}: Control={control_mean:.4f}, Treat={treat_mean:.4f}, Diff={diff:.4f}")
    
    # Check 2: Alternative specification with covariates (if available)
    print("\n2. Placeholder for Additional Checks:")
    print("   - Consider adding municipality fixed effects")
    print("   - Consider adding time-varying covariates")
    print("   - Consider clustering standard errors by municipality")
    print("   - Consider placebo tests in pre-period")

# Run robustness checks
if df is not None:
    run_robustness_checks(df)

## 9. Export Results

In [ ]:
def export_detailed_results(results_dict, filename='Results_1_MDHI_DiD.xlsx'):
    """
    Export detailed regression results to Excel
    """
    try:
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            # Export summary statistics
            if df is not None:
                df.describe().to_excel(writer, sheet_name='Summary_Stats')
            
            # Export each regression result
            for label, result in results_dict.items():
                if result is not None:
                    # Create a DataFrame with coefficients
                    coef_df = pd.DataFrame({
                        'Coefficient': result.params,
                        'Std. Error': result.bse,
                        't-value': result.tvalues,
                        'P-value': result.pvalues,
                        '95% CI Lower': result.conf_int()[0],
                        '95% CI Upper': result.conf_int()[1],
                        'R-squared': result.rsquared
                    })
                    coef_df.to_excel(writer, sheet_name=label[:31])  # Excel sheet name limit
        
        print(f"\nDetailed results exported to '{filename}'")
        
    except Exception as e:
        print(f"Error exporting results: {e}")

# Export results if available
if 'results' in locals() and results:
    export_detailed_results(results)

print("\n" + "=" * 60)
print("ANALYSIS COMPLETE")
print("=" * 60)